<a href="https://colab.research.google.com/github/saakshi20/Linear_Regression_Health_Costs_Calculator/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Encode categorical variables
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

# Split into train (80%) and test (20%) sets
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Separate labels (expenses) from features
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
# Normalize the data
train_stats = train_dataset.describe().transpose()

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Input(shape=[len(train_dataset.keys())]),  # Use Input layer explicitly
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae', 'mse']
    )

    return model

# Build the model
model = build_model()
print("Model built successfully! ✅")


In [ ]:
# Evaluate the model
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))


In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale the datasets
scaler = StandardScaler()

# Fit the scaler on the training dataset
train_dataset_scaled = scaler.fit_transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

# Convert the scaled datasets back into DataFrames
train_dataset = pd.DataFrame(train_dataset_scaled, columns=train_dataset.columns)
test_dataset = pd.DataFrame(test_dataset_scaled, columns=test_dataset.columns)

print("✅ Features scaled successfully!")


In [ ]:
def build_optimized_model():
    model = keras.Sequential([
        layers.Input(shape=[len(train_dataset.keys())]),  # Input layer

        # Hidden layers with BatchNorm and Dropout
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        layers.Dense(1)  # Output layer
    ])

    # Use Adam optimizer with smaller learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae', 'mse']
    )

    return model

# Build the optimized model
model = build_optimized_model()
print("✅ Optimized model built successfully!")


In [ ]:
# 🛠️ Evaluate the model
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print(f"✅ Testing set Mean Abs Error: {mae:.2f} expenses")


In [ ]:
# 📈 Build the optimized model
from tensorflow.keras.regularizers import l2

def build_optimized_model():
    model = keras.Sequential([
        layers.Input(shape=[len(train_dataset.keys())]),
        layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
        layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Switch to Adam

    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae', 'mse']
    )

    return model

# ✅ Build the new model
model = build_optimized_model()

# 🔥 Train with EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    train_dataset, train_labels,
    epochs=300,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


In [ ]:
# 🛠️ Evaluate the model
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
print(f"✅ Testing set Mean Abs Error: {mae:.2f} expenses")


In [ ]:
# 🔥 Plot predictions
test_predictions = model.predict(test_dataset).flatten()

plt.figure(figsize=(8, 8))
plt.scatter(test_labels, test_predictions, alpha=0.5, color='blue', label='Predictions')
plt.xlabel('True Values (expenses)')
plt.ylabel('Predicted Values (expenses)')
plt.title('Model Predictions vs. True Expenses')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, color='red', label='Ideal Line')  # Ideal line
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# 🔥 Plot predictions
test_predictions = model.predict(test_dataset).flatten()

plt.figure(figsize=(8, 8))
plt.scatter(test_labels, test_predictions, alpha=0.5, color='blue', label='Predictions')
plt.xlabel('True Values (expenses)')
plt.ylabel('Predicted Values (expenses)')
plt.title('Model Predictions vs. True Expenses')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, color='red', label='Ideal Line')  # Ideal line
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
